Importing libaries and setting out and defining the algerbraic symbols used in the analytic part of the code



In [29]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

INITIAL_RANGE_PARAMETER = 0.01 # In [fm^-2]
V_0 = 40 # In [MeV, corresponds to the algerbraic symbol V0)
BETA = 0.2 # In [fm^-2], need to check value
REDUCED_MASS = 10/11 * 931.5 # In [units], need to update value and units (10/11 A in MevV?)
ORBITAL_QUANTUM_NUMBER = 0 # In units of hbar, may need to take as a paramter for non ground states? May be more to look at with spin orbit coupling.
HBAR = 1 # In natrul units, also true for c but this does not appear in any algerbraic expresions
SUM_LIMIT = 4 # Determines the number of gaussians we exapand are wave function to

r, μ, β, V0, α, α1, α2, l, hbar = sympy.symbols('r μ β V0 α α1 α2 l hbar')

Where: r is the radius
       μ is the reduced mass
       β is the potential parameter
       V0 is the depth of the potential well
       α is the range parameter for the gaussian wavefunction
       l is the orbital angular momentum qunatum number (we take l=0 to start with)
       hbar is the reduced plancks constant

The gaussian wavefunction, gaussian potential, and kinetic energy operator are defined below 

In [30]:
def gaussian_wave_function(r, α, l):
    """
    Defines the form of the Gaussian wavefunctions used in the series expansion of the wavefunction in terms of the radius r,
    the orbital angular momentum, l, and a range parameter, α_i.

    Parameters
    ----------
    r : sympy variable
    r : sympy variable characterising the gaussian
    l: intger, the orbital angular momentum quantum number

    Returns
    -------
    float the value of the gaussian wavefunction at the radius r
    """
    return r**(l) * sympy.exp(- α * r**2)


def gaussian_potential(r, V0, β):
    """
    Defines the spherically symmetric gaussian potential as a function of radius we use as a 'first guess'
    to make our integrals analytic. Given as V = V_0 * e**(-beta * radius**2)

    Parameters
    ----------
    radius : float
    parameters : 1d array containg the fixed parameters for our potential. In the order [V_0, beta] (see slides/supplimentary notes) 

    Returns
    -------
    float with the value of the potential at a radius r

    """
    return V0 * sympy.exp(-1 * β * r**2)


def kinetic_energy_operator(r, wavefunction, α, hbar, μ, l):
    """
    Defines the kinetic energy operator for our system: '-\frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{l(l+1\hbar^2}{2\mu r^2}'

    Parameters
    ----------
    r : float
    wavefunction : sympy expression describing the wavefunction being operated on
    α : float characterising the gaussian
    hbar: float, the reduced plancks constant
    μ: float, the reduced mass
    l: intger, the orbital angular momentum quantum number

    Returns
    -------
    sympy expression describing the kinetic energy term in the eigenvalue problem

    """
    return - hbar**2 / (2 * μ) * sympy.diff(wavefunction(r, α, l), r, r) + (l * (l + 1) * hbar**2) / (2 * μ * r**2) * wavefunction(r, α, l)

The matrix elements are defined as

In [31]:
def N_ij(r, wavefunction_1, wavefunction_2, α1, α2, l):
    """
    Finds an expression for the overlap term in the eigenvalue problem.
    Evaluates the expression  N_{ij} = \int_0^{\infty}\phi_i(r)\phi_j(r)dr

    Parameters
    ----------
    r : sympy algerba symbol?
    wavefunction_1 : an expression for the lefthand wavefunctions in the system
    wavefunction_2 : an expression for the righthand wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction

    Returns
    -------
    sympy expression describing the overlap term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(r, α, l).subs(α, α1) * wavefunction_2(r, α, l).subs(α, α2), (r, 0, sympy.oo))


def T_ij(r, wavefunction_1, wavefunction_2, α1, α2, hbar, μ, l):
    """
    Finds an expression for the kinetic energy term in the eigenvalue problem.
    Evaluates the expression  T_{ij} = \int_0^{\infty}\phi_i(r)(-\frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{l(l+1\hbar^2}{2\mu r^2})\phi_j(r)dr

    Parameters
    ----------
    radius : float
    wavefunction_1 : an expression for one of the two wavefunctions in the system
    wavefunction_2 : an expression for one of the two wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction

    Returns
    -------
    sympy expression describing the kinetic energy term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(r, α, l).subs(α, α1) * kinetic_energy_operator(r, wavefunction_2, α, hbar, μ, l).subs(α, α2),
                           (r, 0, sympy.oo))


def gaussian_V_ij(r, wavefunction_1, wavefunction_2, α1, α2, potential_function, V0, β, l):
    """
    Finds an expression for the gaussian potential energy term in the eigenvalue problem.
    Evaluates the expression  V_{ij} = \int_0^{\infty}\phi_i(r)V(r)\phi_j(r)dr

    Parameters
    ----------
    radius : float
    wavefunction_1 : an expression for one of the two wavefunctions in the system
    wavefunction_2 : an expression for one of the two wavefunctions in the system
    range_parameter_1 : float, range parameter associated with the first wavefunction
    range_parameter_2 : float, range parameter associated with the second wavefunction
    potential_function : sympy expression detailing the form of the potential

    Returns
    -------
    sympy expression describing the potential energy term in the eigenvalue problem

    """
    return sympy.integrate(wavefunction_1(r, α, l).subs(α, α1) * potential_function(r, V0, β) * wavefunction_2(r, α, l).subs(α, α2),
                           (r, 0, sympy.oo))

Using these we can now generate our matricies to be solved
want to change the kinetic energy operator into the by parts method (integrating the derivative of both)

In [32]:
def matrix_generation(size, orbital_quantum_number, beta, reduced_mass=REDUCED_MASS ,reduced_planks_constant=HBAR):
    h_matrix = np.zeros(shape=(size, size))
    n_matrix = np.zeros(shape=(size, size))
    i = 0 # these count the elements of the matricies created
    j = 0

    while i < size:
        i_range_parameter = next_range_parameter(i)
        while j < size:
            j_range_parameter = next_range_parameter(j)
            h_matrix[i, j] = T_ij(r, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter, hbar, μ, l).subs(
                {α1:i_range_parameter, α2:j_range_parameter, l:orbital_quantum_number,
                 hbar:reduced_planks_constant, μ:reduced_mass, l:orbital_quantum_number}
            ) + gaussian_V_ij(r, gaussian_wave_function, gaussian_wave_function, α1, α2, gaussian_potential, V0, β, l).subs({
                α1:i_range_parameter, α2:j_range_parameter, l:orbital_quantum_number, V0:V_0, β:beta, l:orbital_quantum_number})
            h_matrix[j, i] = h_matrix[i, j]
            n_matrix[i, j] = N_ij(r, gaussian_wave_function, gaussian_wave_function, α1, α2, l).subs({
                α1:i_range_parameter, α2:j_range_parameter, l:orbital_quantum_number})
            n_matrix[j, i] = n_matrix[i, j]
            print('row', i+1, 'column', j+1, 'complete')
            j += 1
        i += 1
        j = i

    return h_matrix, n_matrix


def next_range_parameter(i, initial_range_parameter=INITIAL_RANGE_PARAMETER, geometric_progression_number=2):
    """
    Finds the next range parameter given the previous and initial range parameters.
    Currently using a simple geometric series to determine range parameters.
    Chose geometric basis parameters $\alpha_i = \alpha_1a^{i-1}$ with initial parameters $\alpha_1 = 0.01, a=2$

    Parameters
    ----------
    i : int detailing the iteration number

    Returns
    -------
    new_range_parameter: float

    """
    new_range_parameter = initial_range_parameter * geometric_progression_number**(i)
    return new_range_parameter

Blahdy blahdy blah

In [33]:
h_matrix, n_matrix = matrix_generation(SUM_LIMIT, ORBITAL_QUANTUM_NUMBER, BETA)
print(h_matrix)
print(n_matrix)

row 1 column 1 complete
row 1 column 2 complete
row 1 column 3 complete
row 1 column 4 complete
row 2 column 2 complete
row 2 column 3 complete
row 2 column 4 complete
row 3 column 3 complete
row 3 column 4 complete
row 4 column 4 complete
[[75.57772314 73.91647551 70.89819148 65.82731446]
 [73.91647551 72.36017778 69.52141676 64.7209167 ]
 [70.89819148 69.52141676 66.99253257 62.66578743]
 [65.82731446 64.7209167  62.66578743 59.08189968]]
[[6.26657069 5.11663354 3.9633273  2.95408975]
 [5.11663354 4.43113463 3.61800627 2.80249561]
 [3.9633273  3.61800627 3.13328534 2.55831677]
 [2.95408975 2.80249561 2.55831677 2.21556731]]
